In [2]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import torch

from utils.create_predwin import create_predwin
import os

In [10]:
datapath = os.path.join(os.getcwd(), 'data')
filenames = os.listdir(datapath)

### Clean wind speed & direction dataset

In [13]:
df_list = []
for name in filenames:
    if 'csv' in name and 'annual_energy' not in name:
        temp = pd.read_csv(os.path.join(datapath, name), skiprows=[0,1,2])
        temp = temp.set_index('Time')
        temp.columns = [name[:-4] + '_s', name[:-4] + '_d']
        df_list.append(temp)

In [14]:
wind_df = pd.concat(df_list, axis=1)

In [17]:
wind_df.to_pickle('data/raw_wind_timeseries.pkl')

### Clean energy dataset

In [153]:
column_translate = ['Perimeter', 'Nature', 'Date', 'Hour', 'Consumption', 
                   'Thermal', 'Nuclear', 'Wind', 'Solar', 'Hydraulic',
                   'Pumping', 'Bioenergy', 'ech_physiques']

In [155]:
df_list = []

for name in filenames: 
    if 'annual_energy' in name:
        try: 
            temp = pd.read_csv(os.path.join(datapath, name), encoding= 'unicode_escape')
            temp.columns = column_translate
        except ValueError:
            print('error in ' + name)
            continue
        temp = temp.drop(['Perimeter', 'Nature'], axis=1)
        temp = temp.replace(['-','ND'], np.NaN)
        temp = temp.set_index(['Date', 'Hour'])
        df_list.append(temp)

error in annual_energy_2020.csv


In [159]:
energy_df = pd.concat(df_list,axis=0)

In [161]:
energy_df.to_pickle('data/raw_energy_timeseries.pkl')

In [2]:
energy_2020 = pd.read_csv('data/annual_energy_2020.csv', encoding= 'unicode_escape')

/home/kristoforus/miniconda3/envs/deeplearning/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (4,5,7,8,9,11,12,51,55,57,59,61) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
check = pd.read_pickle('raw_energy_timeseries.pkl')

### Combine datasets

In [404]:
energy = pd.read_pickle('data/raw_energy_timeseries.pkl')
wind = pd.read_pickle('data/raw_wind_timeseries.pkl')
energy_2020 = pd.read_csv('data/annual_energy_2020.csv', encoding= 'unicode_escape')

/home/kristoforus/miniconda3/envs/deeplearning/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (4,5,7,8,9,11,12,51,55,57,59,61) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [405]:
# Combining Wind Energy Data (Only) from 2020
wind_energy = energy[['Wind']].reset_index()
wind_energy_2020 = energy_2020[['Date', 'Heures', 'Eolien']]
wind_energy_2020.columns = ['Date', 'Hour', 'Wind']
wind_energy_com = pd.concat((wind_energy, wind_energy_2020), axis=0).replace('ND', np.nan)

## Taking hourly means 
hours = np.tile(np.repeat(np.arange(24), 4), len(wind_energy_com['Date'].unique()))
wind_energy_com.loc[:, 'hours'] = hours
wind_energy_com = wind_energy_com.drop(['Hour'], axis=1).set_index(['Date', 'hours'])
wind_energy_com['Wind'] = wind_energy_com['Wind'].astype('float')
wind_energy_com = wind_energy_com.groupby(['Date', 'hours']).agg('mean').reset_index()
wind_energy_com['Date'] = wind_energy_com['Date'].astype('datetime64').dt.date

In [406]:
# Break up hour and date in wind dataset
wind.index.name = 'Date'
wind = wind.reset_index()
wind['Date'] = wind['Date'].astype('datetime64')
wind['hours'] = wind['Date'].dt.hour
wind['Date'] = wind['Date'].dt.date

In [407]:
# Combine the two datasets and impute
combined = pd.merge(wind_energy_com, wind, how='left', on=['Date', 'hours'])
combined = combined.sort_values(['Date', 'hours']).set_index(['Date', 'hours'])
combined = combined.groupby(['Date']).fillna(method='ffill')
combined = combined.groupby(['Date']).fillna(method='bfill')
combined = combined.dropna() 

In [415]:
# Save combined, cleaned data 
combined.to_pickle('data/combined_timeseries.pkl')